In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import numpy as np, pandas as pd
import os
import datetime
import csv
import time

In [ ]:
'''两岸四地的国际论文合作是PUB,跟体院排名不一样，因此，本程序不可用'''

In [2]:
#读入csv中的数据，（导入检索式）
a = []
csv_reader=csv.reader(open(r'C:\Users\rk\Desktop\UK.csv'))
for row in csv_reader:
    a.append(row)#将读入的数据赋值给列表a
#del a[0]#删除表头
c='AND DT=(Article or Review)'
for lst in a:
    lst[1]=lst[1]+' '+c
count=0

In [3]:
a[0]

['拉夫堡大学',
 'AD=((Loughborough Univ or Univ Loughborough) same sport* same England) AND DT=(Article or Review)']

In [4]:
del a[0]

In [5]:
a[0]

['利物浦约翰摩尔斯大学',
 'AD=((Liverpool John Moores Univ or Liverpool JMU) same sport* same England) AND DT=(Article or Review)']

In [6]:
a[0][1]

'AD=((Liverpool John Moores Univ or Liverpool JMU) same sport* same England) AND DT=(Article or Review)'

In [2]:
########################服务器崩溃后，从此处运行##########################################
browser = webdriver.Chrome('C:/Users/rk/Desktop/chromedriver.exe') #锁定浏览器


In [3]:
######################此处与下面重复，但程序刚开始运行时，及服务器崩溃后需要重新运行###################
browser.get("http://apps.webofknowledge.com/WOS_AdvancedSearch_input.do?SID=8EJqZrebXFQVSmd9PsP&product=WOS&search_mode=AdvancedSearch")

In [ ]:
#此处是程序运行出问题，无法进行，只能清除所有，重新启动后将count设置为之前的，接着运行，因为已经保存TXT，所以放心
count=294 
#再三强调，不能随意执行这一步，除非程序重启

In [16]:
def delete(history_record):
    '''删除记录，满足history_record条即删掉除第一条之外的记录'''
    if int(browser.find_element_by_class_name('historySetNum').text[2:])==history_record:#删除记录条数
        browser.find_element_by_xpath('/html/body/div[13]/form/table/tbody/tr[1]/th[6]/div[2]/button[1]').click()#全选
        browser.find_element_by_xpath("//input[@type='checkbox'and @name='dSet']").click()#取消第一个
        browser.find_element_by_xpath('/html/body/div[13]/form/table/tbody/tr[1]/th[6]/div[2]/button[2]/i').click()#删除
    elif int(browser.find_element_by_class_name('historySetNum').text[2:])==history_record+1:
        browser.find_element_by_xpath('/html/body/div[13]/form/table/tbody/tr[1]/th[6]/div[2]/button[1]').click()#全选
        browser.find_element_by_xpath("//input[@type='checkbox'and @name='dSet']").click()#取消第一个
        browser.find_element_by_xpath('/html/body/div[13]/form/table/tbody/tr[1]/th[6]/div[2]/button[2]/i').click()#删除

In [7]:
def search(item): 
    '''向搜索框里输入检索式'''
    browser.find_element_by_id('value(input1)').clear()#清除文本框里的内容
    browser.find_element_by_id('value(input1)').send_keys(item)#往文本框里输入搜索文本（输入检索式）
    browser.find_element_by_id('searchButton')#定位搜索按钮
    browser.find_element_by_id('searchButton').click()#点击搜素按钮

In [8]:
def write_record():
    '''保存每次检索结果，防止程序崩溃，浏览器崩溃，服务器奔溃'''
    with (open(r'C:\Users\rk\Desktop\papers1.txt','a+')) as file:
            str='、'.join(item)
            file.write(str+"\n")
            print(item)

In [9]:
def click_all(inputs):
    '''点击所有复选框'''
    for i in inputs:
            if i.get_attribute('type')== 'checkbox':#点击所有复选框
                i.click()

In [10]:
def UKS(input1,input2):
    '''判断是否属于英国地区'''
    UK=('WALES','ENGLAND','SCOTLAND','NORTH IRELAND')#英国的四个地区
    flag=0
    get_txt=input2[0].text
    country=get_txt[:get_txt.find('(')]
    reg=country.strip()
    for region in UK:#判断地区是否属于英国
        if region == reg:
            flag=1
            break   
    if flag==1:
        for i in range(0,len(input1)):
            get_txt=input2[i].text
            country=get_txt[:get_txt.find('(')]
            reg=country.strip() 
            for j in range(0,len(UK)):
                if reg==UK[j]:
                    input2[i].click()#如果属于则再次点击复选框，以取消选项
    else:input1[0].click()#再次点击第一个复选框

In [19]:
##########################程序中断后，从此处运行#####################################
#for循环中的count，进入循环后就不再改变，若想每次循环都改变，只有赋值，使用break强制
#循环结束退出，然后利用while继续循环，别忘了对while设置退出条件
print(count)
for item in a[count:]:
    browser.get("http://apps.webofknowledge.com/WOS_AdvancedSearch_input.do?SID=8EJqZrebXFQVSmd9PsP&product=WOS&search_mode=AdvancedSearch")
    
    try:
        delete(2)
    except:
        pass
    
    search(item[1])#往文本框里输入搜索文本（输入检索式）
    
    d=browser.find_element_by_xpath('//*[@id="client_error_input_message"]').text#定位异常信息
    if len(d)==0:#判断检索式是否异常（如果正常运行，则不会弹出异常，那么异常信息len为零）
        papers_number=browser.find_element_by_class_name('historyResults').text
        #如果没有异常，则定位第一个数据，因为每搜索一次，数据系列就增加一次，无法用ID定位，只能广泛定位，自动匹配第一个数据
        
        item.append(papers_number)#将找到的数据拼接到原数据的后面
    else: 
        item.append('NA')#搜索异常的时候讲自定义的异常标识拼接到原数据后
        item.append('NA')
        write_record()
        count+=1
        print(count)
        continue
        
    #查询国际论文合作数会导致历史记录以公差为2递增，故前后设置了两次删除 
    delete(2)
        
    #获取除自引的被引频数
    browser.find_element_by_class_name('historyResults').click() 
    browser.find_element_by_xpath('//*[@id="refine_form"]/div[4]/a').click()#点击查看全部选项
    d=browser.find_element_by_xpath('//*[@id="CountryTerritory_tr"]').text
    if len(d)==0:
        browser.find_element_by_xpath('//*[@id="CountryTerritory"]/h4').click()#如果国家地区没有展开，则点击国家地区
    time.sleep(4)
    try:
        browser.find_element_by_xpath('//*[@id="CountryTerritory_tr"]/div[2]/a').click()#点击更多选项分类
        inputs=browser.find_elements_by_xpath('//*[@id="CountryTerritory_raMore_tr"]/td/table/tbody//input')#查找所有复选项
        
        click_all(inputs) 
       
        inputs1=browser.find_elements_by_xpath('//*[@id="CountryTerritory_raMore_tr"]/td/table/tbody//label')
        
        UKS(inputs,inputs1)
        
        browser.find_element_by_xpath('//*[@id="raMore"]/table/tbody/tr[6]/td/table/tbody/tr/td[1]/div/button').click()#点击精炼
        IC_number=browser.find_element_by_xpath('//*[@id="hitCount.top"]').text#提取国际合作论文数
        item.append(IC_number)
        #保存每条记录，浏览器，程序及服务器会崩溃，以防重新执行浪费时间
        write_record()
        count+=1
        print(count)
    except:
        inputs2=browser.find_elements_by_xpath('//*[@id="CountryTerritory"]/div/div/div/div/input')#查找所有复选项
        if len(inputs2)==1:
            item.append("zero")
            write_record()
            count+=1
            print(count)
            continue
        else:
            click_all(inputs2)
        inputs3=browser.find_elements_by_xpath('//*[@id="CountryTerritory_raMore_tr"]/td/table/tbody//label')
        
        UKS(inputs2,inputs3)
        
        browser.find_element_by_xpath('//*[@id="CountryTerritory"]/div/div/button[1]').click()#点击精炼
        IC_number=browser.find_element_by_xpath('//*[@id="hitCount.top"]').text#提取国际合作论文数
        item.append(IC_number) 
        write_record()
        count+=1
        print(count)
    

10
['卡迪夫城市大学', 'AD=((Cardiff Metropolitan Univ or Univ wales inst) same sport* same Wales) AND DT=(Article or Review)', '215', '105']
11
['贝德福德大学', 'AD=(Univ Bedfordshire same phys act same England) AND DT=(Article or Review)', '67', '24']
12
['谢菲尔德哈勒姆大学', 'AD=(Sheffield Hallam Univ same (sport* or phys act) same England) AND DT=(Article or Review)', '212', '108']
13
['诺森比亚大学', 'AD=(Northumbria Univ same sport* same England) AND DT=(Article or Review)', '162', '73']
14


KeyboardInterrupt: 

In [ ]:
result=pd.DataFrame(np.array(a))#将pd.DataFrame(np.array(myarray))写入list
result.to_csv("C:/Users/rk/Desktop/IC.csv",index=False,encoding='ANSI')
result